##### U-Net

- Image segmentation(predict label for every pixel in an image)
<img src="../images/image segmentation.png" style="width:500px;height:250;">
- U-Net architecture
    <img src="../images/unet architecture.png" style="width:700px;height:400;">
    - Fully Convolutional Network(FCN)
        - replaces dense layers with a transposed convolution layer that upsamples the feature map back to the size of the original input image since dense layers destroy spatial information